In [1]:
# Import libraries
using Turing
using LinearAlgebra
using Distributions
using MultivariateStats
import MultivariateStats: reconstruct
using GaussianProcesses
using StatsBase
using Statistics
using Suppressor
using JLD2
using CSV
using DataFrames, DataFramesMeta
using SplitApplyCombine
using KernelFunctions
using MCMCChains
using PyCall
using PyPlot
using Printf
import PyCall.pyfunction


# Access the matplotlib module
matplotlib = pyimport("matplotlib")
seaborn = pyimport("seaborn")
os = pyimport("os")
pyimport("scienceplots")
np = pyimport("numpy")

plt.style.use(["default","science","no-latex"])
using StatsPlots

seaborn.color_palette("colorblind")

PyCall.pygui(:tk)

# Set a seed for reproducibility
using Random
Random.seed!(11);

FONTSIZE=20.5;

In [2]:
function tuple_to_idx(x,y)
    return ( (3*x - 3) + y )
end

tuple_to_idx (generic function with 1 method)

In [3]:
function get_up_low(post_dict, upper_quant, lower_quant)
    p = [upper_quant,lower_quant]
    quantiles = zeros(19, 6, 2)
    sorted_keys = sort(collect(keys(post_dict)))
    for i in 1:19
        mat = post_dict[ sorted_keys[i] ]
        quantiles[i, 1,:] = quantile( mat[:,1], p )
        quantiles[i, 2,:] = quantile( mat[:,2], p )
        quantiles[i, 3,:] = quantile( mat[:,3], p )
        quantiles[i, 4,:] = quantile( mat[:,4], p )
        quantiles[i, 5,:] = quantile( mat[:,5], p )
        quantiles[i, 6,:] = quantile( mat[:,6], p )
    end
    return quantiles
end



get_up_low (generic function with 1 method)

In [4]:
Realizations = [i for i in 1:100];
#Misconverged realization numbers
m = [4, 15, 20, 44, 47,67,81,82,89,100]
filter!(x -> !(x in m), Realizations)
r_filtered = [string(r) for r in Realizations];

In [ ]:
#CHANGE THIS LINE TO WHERE THE POSTERIORS ARE LOCATED
#path_to_posteriors = "Data/Posterior_Data"
path_to_posteriors = "../../BrookhavenCode/ProjectNotebooks/Sanket_Results_Posteriors"

output_file_name = "../Plots/Parametric_Learning_Plots/Without_Trajectories/parametric_learning.pdf"

title_dict = Dict([
            1 => L"\sigma_{max}", 2 => L"q" , 3 => L"C_{\mu}" ,
            4 => L"C_{\phi}", 5 => L"\gamma_0", 6 => L"\overline{m}" ]) 

fig, ax = PyPlot.subplots(nrows=2, ncols=3, figsize=(18, 11), dpi=300,
                     gridspec_kw=Dict("height_ratios"=> [1, 1], "width_ratios" => [1, 1, 1],
                                    "wspace"=> 0.175, "hspace"=> 0.25))

total_widths = zeros(90,19,6)
sorted_keys = collect(range(2030,step=15,length=19))
#Plot each parameter's learning against last year observed, for each trajectory
    for (iter, r) in enumerate(r_filtered)
        post = JLD2.load("$(path_to_posteriors)/R_$(r)_Posterior_Dict.jld2", "post_data")
        cred_int_95_up_low = get_up_low(post, 0.95, 0.05)
        for i in 1:2
            for j in 1:3
                var_idx = tuple_to_idx(i,j)
                widths = cred_int_95_up_low[:,var_idx,1] .- cred_int_95_up_low[:,var_idx,2]
                #Add this realizations cred int width to the total # 22 Below
                total_widths[iter, :, var_idx] = widths

                #UNCOMMENT LINE BELOW TO ADD TRAJECTORIES TO PLOTS
                #ax[i,j].plot(sorted_keys, widths, marker = "o", markersize= 3, label = "Individual scenario",ls="dashed", color = "black", markerfacecolor="red")
                #ax[i,j].axhline(y=actual_means[var_idx], color = "red", linestyle="dashed", label = L"\theta", lw = 2)
            end
        end
    end

    #Plot each parameter's average learning across the 90 trajectories
    for i in 1:2
        for j in 1:3
            var_idx = tuple_to_idx(i,j)
            current = total_widths[:,:,var_idx]
            #println(size(current))
            mean_widths = vec(mean(current, dims=1))
            #println(length(mean_widths))
            std_widths = vec(std(current, dims=1) )
            upper = mean_widths .+ std_widths
            lower = mean_widths .- std_widths
            ax[i,j].plot(sorted_keys, mean_widths, color="blue",lw=2.5,label="Average across scenarios", zorder = 601)
            ax[i,j].fill_between(sorted_keys, upper, lower, color="red",alpha = 0.3, label="68% credible interval",zorder = 600)
            
            ax[i,j].set_title(title_dict[var_idx],fontsize=1.2*FONTSIZE, pad=10)
            ax[i,j].tick_params(axis="both", which="major", labelsize=FONTSIZE*0.8)
            ax[i,j].set_xlim(2030,2300)
    
        end
    end



handles, labels = ax[1,1].get_legend_handles_labels()
desired_indices = [1,2]
sorted_handles = [handles[i] for i in desired_indices]
sorted_labels = [labels[i] for i in desired_indices]
fig.legend(sorted_handles, sorted_labels, loc="lower center", ncol=6, fontsize=FONTSIZE,
           bbox_to_anchor=(0.512, 0.91), bbox_transform=PyPlot.gcf().transFigure,
           frameon="True", framealpha=1)

fig.supxlabel("Year of last observation", fontsize=FONTSIZE*1.2)
fig.supylabel("90% uncertainty range", fontsize=FONTSIZE*1.2)

plt.savefig(output_file_name)

        